In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ds.data.sets import load_sets_v2
import joblib
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer

2023-07-11 01:02:14.269558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(55)

In [4]:
datasets = joblib.load("../data/processed/datasetsjmone")

In [5]:
drops = [
    "review_overall",
    "beer_name",
]
for data in datasets:
    if isinstance(data, pd.DataFrame):
        for column in drops:
            if column in data.columns:
                data.drop(columns=column, inplace=True)

In [6]:
x_train, y_train, x_test, y_test, x_validation, y_validation = datasets

In [7]:
x_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
590715,goose island beer co,5.0,4.5,4.0,5.0,13.00
1514329,hinterland brewery restaurant,4.0,4.0,3.5,3.5,6.85
272592,flying dog brewery,3.5,3.0,3.5,3.5,5.50
894213,stone brewing co,4.0,4.5,4.0,4.0,7.20
174214,long trail brewing co,4.0,4.0,4.0,4.0,8.60
...,...,...,...,...,...,...
759715,widmer brothers brewing company,3.5,4.5,3.0,3.5,9.40
618842,harpoon brewery,3.5,3.5,3.5,3.5,4.30
948973,bells brewery inc,3.5,3.5,3.5,3.5,5.20
678833,moorhouses brewery burnley ltd,3.5,4.0,3.0,3.5,3.40


In [8]:
x_train = x_train.fillna(-9)
x_test = x_test.fillna(-9)

In [9]:
BATCH_SIZE = 10000

In [10]:
df = x_train


In [11]:
y = y_train
y_vocab = sorted(set(y))

In [12]:
y_tensor = tf.convert_to_tensor(y)

2023-07-11 01:02:16.457127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 01:02:16.459855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 01:02:16.459990: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
target_lookup = tf.keras.layers.StringLookup(vocabulary=y_vocab, output_mode='one_hot')
target = target_lookup(y_tensor)

In [14]:
ds = tf.data.Dataset.from_tensor_slices((dict(df), target))
ds = ds.batch(BATCH_SIZE)


In [15]:
y_test_tensor = tf.convert_to_tensor(y_test)
y_test_encoded = target_lookup(y_test_tensor)
test = tf.data.Dataset.from_tensor_slices((dict(x_test), y_test_encoded))
test = test.batch(BATCH_SIZE)

In [16]:
# #compute imputer mean
# mask = tf.where(tf.math.is_nan(tensor) , 0. , tensor)
# mask_norm = tf.reduce_sum(tf.clip_by_value(mask, 0., 1.),axis=0)
# imp_mean = tf.math.divide(tf.reduce_sum(mask, axis=0), mask_norm)

# #transform
# tf.where(tf.math.is_nan(X) , imp_mean , X)

In [17]:
numeric_feature_names = [
    # "review_overall",
    "review_aroma",
    "review_appearance",
    "review_palate",
    "review_taste",
]
numeric_features = df[numeric_feature_names]

categorical_feature_names = [
    "brewery_name"
]

In [18]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [19]:
inputs = {}
preprocessed = []

In [20]:
for name, column in x_train.items():
  if type(column[0]) == str:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

In [21]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

In [22]:
numeric_inputs = {}
for name in numeric_feature_names:
  numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

preprocessed

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization')>]

In [23]:
for name in categorical_feature_names:
  vocab = sorted(set(df[name]))

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x)

In [24]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocessor = tf.keras.Model(inputs, preprocesssed_result)
tf.keras.utils.plot_model(preprocessor, rankdir="LR", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [138]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1= tf.keras.layers.Dense(4096, activation='relu')
    self.dense2 = tf.keras.layers.Dense(2048, activation='relu')
    self.dense3 = tf.keras.layers.Dense(4096, activation='relu')
    self.dense4 = tf.keras.layers.Dense(1024, activation='relu')
    self.dense5 = tf.keras.layers.Dense(2048, activation='relu')
    self.attn1 = tf.keras.layers.Attention()
    self.attn2 = tf.keras.layers.Attention()
    self.dropout = tf.keras.layers.Dropout(0.5)
    self.outputlyr = tf.keras.layers.Dense(target_lookup.vocabulary_size(), activation='softmax')
    self.flatten = tf.keras.layers.Flatten()
    self.mask = tf.keras.layers.Masking(mask_value=0.0)
    
  def call(self, inputs, training=False):
    # x = self.flatten(inputs)
    x = self.dense1(inputs)
    # x = self.attn1(([x, x]))
    x = self.dense2(x)
    x = self.dense3(x)

    # if training:
    #   x = self.dropout(x, training=training)
    x = self.dense4(x)
    # x = self.dense5(x)

    return self.outputlyr(x)

body = MyModel()

In [139]:
# body = tf.keras.Sequential([
#   tf.keras.layers.Dense(512, activation='relu'),
#   tf.keras.layers.Dense(256, activation='relu'),
#   tf.keras.layers.Dense(32, activation='relu'),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(32, activation='relu'),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(target_lookup.vocabulary_size(), activation='softmax')
# ])

In [140]:
x = preprocessor(inputs)
x

<KerasTensor: shape=(None, 5862) dtype=float32 (created by layer 'model')>

In [141]:
result = body(x)
result

<KerasTensor: shape=(None, 105) dtype=float32 (created by layer 'my_model_20')>

In [142]:
model = tf.keras.Model(inputs, result)

model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

In [143]:
history = model.fit(
    ds,
    epochs=200,
    batch_size=BATCH_SIZE,
    validation_data=test
    )

Epoch 1/200
137/137 [==============================] - 37s 264ms/step - loss: 3.0738 - accuracy: 0.2246 - val_loss: 2.4516 - val_accuracy: 0.3069
Epoch 2/200
137/137 [==============================] - 36s 263ms/step - loss: 2.3250 - accuracy: 0.3163 - val_loss: 2.3084 - val_accuracy: 0.3111
Epoch 3/200
137/137 [==============================] - 36s 263ms/step - loss: 2.2261 - accuracy: 0.3240 - val_loss: 2.2650 - val_accuracy: 0.3154
Epoch 4/200
137/137 [==============================] - 36s 263ms/step - loss: 2.1810 - accuracy: 0.3294 - val_loss: 2.2664 - val_accuracy: 0.3129
Epoch 5/200
137/137 [==============================] - 36s 263ms/step - loss: 2.1538 - accuracy: 0.3333 - val_loss: 2.2554 - val_accuracy: 0.3152
Epoch 6/200
137/137 [==============================] - 36s 264ms/step - loss: 2.1321 - accuracy: 0.3369 - val_loss: 2.2554 - val_accuracy: 0.3176
Epoch 7/200
137/137 [==============================] - 36s 263ms/step - loss: 2.1133 - accuracy: 0.3406 - val_loss: 2.2665 -